In [6]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import norm
from scipy.integrate import odeint
import pprint
from ipywidgets import interact, interactive, fixed, interact_manual, jslink
import ipywidgets as widgets
import ipyvolume as ipv
import ipyvolume.pylab as p3
import traitlets
import time, sys
from IPython.core.display import clear_output

In [229]:
# define the atoms in the chain (this test will only use a 3 atom chain)
r1 = (0, 0, 1)
r2 = (2, 1, 1)
r3 = (2, 2, 1)

floaters = np.hstack((r1, r2, r3))

m = 3             # total atoms in the chain

In [230]:
# define parameters
beta = 1

In [231]:
# define time interval
t = np.linspace(0,6,10)

In [232]:
# bending force DE system
def bending_force(floaters, t):
    
    current_bending_force = np.zeros((m,3))
    total_bending_force = np.zeros(m)
    
    # loop for computing bending force
    for i in range(1,m-1):          # loop through each atom not on the end of the chain and compute the bending force on each
        
        print(i)
        # define the pieces of the bending energy equation
        sx_n = floaters[i*3] - floaters[(i-1)*3]
        sx_n1 = floaters[(i+1)*3] - floaters[i*3]

        sy_n = floaters[i*3 + 1] - floaters[(i-1)*3 + 1]
        sy_n1 = floaters[(i+1)*3 + 1] - floaters[i*3 + 1]

        sz_n = floaters[i*3 + 2] - floaters[(i-1)*3 + 2]
        sz_n1 = floaters[(i+1)*3 + 2] - floaters[i*3 + 2]

        A = norm((sx_n, sy_n, sz_n))*norm((sx_n1, sy_n1, sz_n1))

        B = np.dot((sx_n, sy_n, sz_n),(sx_n1, sy_n1, sz_n1))

        partial_common = .5/A
        
        # partials of A wrt x for atom n, n-1 and n+1, respectively
        Axn = partial_common*(2*(sx_n)*(sx_n1**2) - 2*(sx_n1)*(sx_n**2) + 2*(sx_n)*(sy_n1**2) + 2*(sx_n)*(sz_n1**2) - 2*(sx_n1)*(sy_n**2) - 2*(sx_n1)*(sz_n**2))
        Axn0 = partial_common*(-2*(sx_n)*(sx_n1**2) - 2*(sx_n)*(sy_n1**2) - 2*(sx_n)*(sz_n1**2))
        Axn1 = partial_common*(2*(sx_n**2)*(sx_n1) + 2*(sy_n**2)*(sx_n1) + 2*(sz_n**2)*(sx_n1))
        
        # partials of A wrt y for atom n, n-1 and n+1, respectively
        Ayn = partial_common*(-2*(sx_n**2)*(sy_n1) + 2*(sy_n)*(sx_n1**2) + 2*(sy_n)*(sy_n1**2) - 2*(sy_n**2)*(sy_n1) + 2*(sy_n)*(sz_n1**2) - 2*(sz_n**2)*(sy_n1))
        Ayn0 = partial_common*(-2*(sy_n)*(sx_n1**2) - 2*(sy_n)*(sy_n1**2) - 2*(sy_n)*(sz_n1**2))
        Ayn1 = partial_common*(2*(sy_n**2)*(sy_n1) + 2*(sz_n**2)*(sy_n1) + 2*(sx_n**2)*(sy_n1))
        
        # partials of A wrt z for atom n, n-1 and n+1, respectively
        Azn = partial_common*(-2*(sx_n**2)*(sz_n1) - 2*(sy_n**2)*(sz_n1) - 2*(sz_n**2)*(sz_n1) + 2*(sz_n)*(sz_n1**2) + 2*(sz_n)*(sx_n1**2) + 2*(sz_n)*(sy_n1**2))
        Azn0 = partial_common*(-2*(sz_n)*(sx_n1**2) -2*(sz_n)*(sy_n1**2) - 2*(sz_n)*(sz_n1**2))
        Azn1 = partial_common*(2*(sz_n1)*(sx_n**2) + 2*(sz_n1)*(sy_n**2) + 2*(sz_n1)*(sz_n**2))
        
        # partials of B wrt x for atom n, n-1 and n+1, respectively
        Bxn = floaters[(i+1)*3] - 2*floaters[i*3] + floaters[(i-1)*3]
        Bxn0 = -floaters[(i+1)*3] + floaters[i*3]
        Bxn1 = floaters[i*3] - floaters[(i-1)*3]

        # partials of B wrt y for atom n, n-1 and n+1, respectively
        Byn = floaters[(i+1)*3 + 1] - 2*floaters[i*3 + 1] + floaters[(i-1)*3 + 1]
        Byn0 = -floaters[(i+1)*3 + 1] + floaters[i*3 + 1]
        Byn1 = floaters[i*3 + 1] - floaters[(i-1)*3 + 1]
        
        # partials of B wrt z for atom n, n-1 and n+1, respectively
        Bzn = floaters[(i+1)*3 + 2] - 2*floaters[i*3 + 2] + floaters[(i-1)*3 + 2]
        Bzn0 = -floaters[(i+1)*3 + 2] + floaters[i*3 + 2]
        Bzn1 = floaters[i*3 + 2] - floaters[(i-1)*3 + 2]
        
        try:
            print("try")
            Eb_common = 4*beta/((A+B)**2)                   # compute common part of bending energy gradient
            if (np.isnan(Eb_common)  or np.isinf(Eb_common) or np.isnan(A)):
                raise ValueError('Divide by zero')
                raise Exception
            current_bending_force[i][0] += Eb_common*(B*Axn - A*Bxn)
            current_bending_force[i][1] += Eb_common*(B*Ayn - A*Byn)
            current_bending_force[i][2] += Eb_common*(B*Azn - A*Bzn)
            
            current_bending_force[i-1][0] += Eb_common*(B*Axn0 - A*Bxn0)
            current_bending_force[i-1][1] += Eb_common*(B*Ayn0 - A*Byn0)
            current_bending_force[i-1][2] += Eb_common*(B*Azn0 - A*Bzn0)
            
            current_bending_force[i+1][0] += Eb_common*(B*Axn1 - A*Bxn1)
            current_bending_force[i+1][1] += Eb_common*(B*Ayn1 - A*Byn1)
            current_bending_force[i+1][2] += Eb_common*(B*Azn1 - A*Bzn1)
            
        except Exception as error:
            print("except")
            print("Caught exception: ", error)
            continue
            

    total_bending_force = np.hstack(current_bending_force)
    print(total_bending_force)
    return total_bending_force

In [233]:
%%time

# solve the system of ODEs to compute changing positions as a result of the bending force acting on the chain
gFlow = odeint(bending_force, floaters, t, atol=1.4e-10)

1
0.22360679775
try
[-0.34164079  0.68328157  0.          2.04984472 -0.68328157  0.
 -1.70820393  0.          0.        ]
1
0.223606284955
try
[ -3.41643980e-01   6.83290898e-01   0.00000000e+00   2.04986976e+00
  -6.83280125e-01   0.00000000e+00  -1.70822578e+00  -1.07728974e-05
   0.00000000e+00]
1
0.223606284956
try
[ -3.41643980e-01   6.83290898e-01   0.00000000e+00   2.04986976e+00
  -6.83280125e-01   0.00000000e+00  -1.70822578e+00  -1.07730317e-05
   0.00000000e+00]
1
0.223605772155
try
[ -3.41647174e-01   6.83300224e-01   0.00000000e+00   2.04989480e+00
  -6.83278678e-01   0.00000000e+00  -1.70824762e+00  -2.15463143e-05
   0.00000000e+00]
1
0.223605772156
try
[ -3.41647174e-01   6.83300224e-01   0.00000000e+00   2.04989480e+00
  -6.83278678e-01   0.00000000e+00  -1.70824762e+00  -2.15464487e-05
   0.00000000e+00]
1
0.223544181018
try
[ -3.42030879e-01   6.84420925e-01   0.00000000e+00   2.05290314e+00
  -6.83103666e-01   0.00000000e+00  -1.71087226e+00  -1.31725880e-03
   0.0

 -17.78039673 -53.87303      0.        ]
1
0.132236384582
try
[ -7.52483246  25.964776     0.          25.73814408  31.67409845   0.
 -18.21331162 -57.63887445   0.        ]
1
0.132236386355
try
[ -7.52483508  25.96478488   0.          25.73814935  31.67411196   0.
 -18.21331427 -57.63889684   0.        ]
1
0.131696747852
try
[ -8.05645059  27.8218789    0.          26.73321501  34.19421554   0.
 -18.67676442 -62.01609445   0.        ]
1
0.131696753663
try
[ -8.056461    27.82191425   0.          26.73323502  34.1942689    0.
 -18.67677402 -62.01618315   0.        ]
1
0.1311328427
try
[ -8.68360604  30.01052915   0.          27.85679975  37.16516359   0.
 -19.1731937  -67.17569274   0.        ]
1
0.131132850635
try
[ -8.68362251  30.01058507   0.          27.85682988  37.16524756   0.
 -19.17320737 -67.17583263   0.        ]
1
0.131237203603
try
[ -8.5614396   29.58436778   0.          27.64195436  36.586626     0.
 -19.08051476 -66.17099379   0.        ]
1
0.131237207637
try
[ -8.5614

1
0.120816134193
try
[ -714.10695236  2422.15210751     0.          -179.88124314  3153.20898154
     0.           893.9881955  -5575.36108905     0.        ]
1
0.120824135416
try
[ -699.55876839  2373.02751439     0.          -171.36315138  3090.00475441
     0.           870.92191977 -5463.0322688      0.        ]
1
0.12082413659
try
[ -699.56303808  2373.04192729     0.          -171.36565787  3090.02340461
     0.           870.92869595 -5463.0653319      0.        ]
1
0.120789728569
try
[ -765.84860548  2596.84146117     0.          -210.69133066  3377.85001957
     0.           976.53993614 -5974.69148075     0.        ]
1
0.120789729256
try
[ -765.85139188  2596.85086445     0.          -210.69302102  3377.86217188
     0.           976.5444129  -5974.71303633     0.        ]
1
0.120753054681
try
[ -848.8240802   2876.90493272     0.          -261.63744353  3737.64866971
     0.          1110.46152373 -6614.55360243     0.        ]
1
0.120753056985
try
[ -848.83580371  2876.9444

try
[ -7297673.53090961  24418304.20149548         0.          -8697734.83311134
  30278103.61325122         0.          15995408.36401824 -54696407.8147467
         0.        ]
1
0.120235699272
try
[ -7300533.56549732  24427871.83757536         0.          -8701185.82211425
  30289955.05176903         0.          16001719.38760684
 -54717826.88934371         0.        ]
1
0.120235745465
try
[ -6910670.09598476  23123661.35085487         0.          -8230876.09673535
  28674415.61135207         0.          15141546.19271759
 -51798076.96220648         0.        ]
1
0.12023574551
try
[ -6911558.59739528  23126633.66749542         0.          -8231947.70401633
  28678097.56649018         0.          15143506.3014135  -51804731.23398545
         0.        ]
1
0.120235625002
try
[ -8004892.32444779  26784137.58589632         0.          -9551394.81555162
  33208535.82945457         0.          17556287.13999788
 -59992673.41535147         0.        ]
1
0.120235625042
try
[ -8005814.0452700

   0.00000000e+00]
1
0.120234454561
try
[ -1.69455335e+16   5.66622112e+16   0.00000000e+00  -2.09452782e+16
   7.00385537e+16   0.00000000e+00   3.78908116e+16  -1.26700765e+17
   0.00000000e+00]
1
0.120234454561
try
[ -2.20029907e+16   7.35732549e+16   0.00000000e+00  -2.71965352e+16
   9.09417973e+16   0.00000000e+00   4.91995259e+16  -1.64515052e+17
   0.00000000e+00]
1
0.120234454561
try
[ -2.21438560e+16   7.40442780e+16   0.00000000e+00  -2.73706515e+16
   9.15240150e+16   0.00000000e+00   4.95145075e+16  -1.65568293e+17
   0.00000000e+00]
1
0.120234454561
try
[ -3.29647876e+16   1.10227133e+17   0.00000000e+00  -4.07458609e+16
   1.36248570e+17   0.00000000e+00   7.37106484e+16  -2.46475702e+17
   0.00000000e+00]
1
0.120234454561
try
[ -3.37633573e+16   1.12897377e+17   0.00000000e+00  -4.17329337e+16
   1.39549179e+17   0.00000000e+00   7.54962910e+16  -2.52446556e+17
   0.00000000e+00]
1
0.120234454561
try
[ -7.52296237e+16   2.51551595e+17   0.00000000e+00  -9.29874553e+16
 

   0.00000000e+00]
1
0.12023445456
try
except
Caught exception:  Divide by zero
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
1
0.12023445456
try
[  2.94656968e+22  -9.85269210e+22   0.00000000e+00   3.64216550e+22
  -1.21786035e+23   0.00000000e+00  -6.58873653e+22   2.20312950e+23
   0.00000000e+00]
1
0.12023445456
try
except
Caught exception:  Divide by zero
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
1
0.120234454666
try
except
Caught exception:  Divide by zero
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
1
0.120234454654
try
except
Caught exception:  Divide by zero
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
1
0.12023445456
try
except
Caught exception:  Divide by zero
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
1
0.120234449547
try
except
Caught exception:  Divide by zero
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
1
0.120234453781
try
except
Caught exception:  Divide by zero
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
1
0.12023445456
try
except
Caught exception:  Divide by zero
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
1
0.1202

/Users/christopher/.anaconda/ipykernel/ipykernel_launcher.py:54: RuntimeWarning: divide by zero encountered in double_scalars
/anaconda/lib/python3.6/site-packages/scipy/integrate/odepack.py:218: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


In [234]:
print(gFlow)

[[  0.00000000e+00   0.00000000e+00   1.00000000e+00   2.00000000e+00
    1.00000000e+00   1.00000000e+00   2.00000000e+00   2.00000000e+00
    1.00000000e+00]
 [ -1.78842430e-01   5.47764947e-01   1.00000000e+00   2.74269711e+00
    1.42148784e+00   1.00000000e+00   1.43614532e+00   1.03074721e+00
    1.00000000e+00]
 [  9.03864388e-17   9.03864388e-17   1.00000000e+00   1.00000000e+00
    1.00000000e+00   1.00000000e+00   3.00000000e+00   3.00000000e+00
    1.00000000e+00]
 [  9.03864388e-17   9.03864388e-17   1.00000000e+00   1.00000000e+00
    1.00000000e+00   1.00000000e+00   3.00000000e+00   3.00000000e+00
    1.00000000e+00]
 [  9.03864388e-17   9.03864388e-17   1.00000000e+00   1.00000000e+00
    1.00000000e+00   1.00000000e+00   3.00000000e+00   3.00000000e+00
    1.00000000e+00]
 [  9.03864388e-17   9.03864388e-17   1.00000000e+00   1.00000000e+00
    1.00000000e+00   1.00000000e+00   3.00000000e+00   3.00000000e+00
    1.00000000e+00]
 [  9.03864388e-17   9.03864388e-17   1.

In [228]:
sx_n = gFlow[-1][3] - gFlow[-1][0]
sx_n1 = gFlow[-1][6] - gFlow[-1][3]

sy_n = gFlow[-1][4] - gFlow[-1][1]
sy_n1 = gFlow[-1][7] - gFlow[-1][4]

sz_n = gFlow[-1][5] - gFlow[-1][2]
sz_n1 = gFlow[-1][8] - gFlow[-1][5]

A = norm((sx_n, sy_n, sz_n))*norm((sx_n1, sy_n1, sz_n1))

B = np.dot((sx_n, sy_n, sz_n),(sx_n1, sy_n1, sz_n1))

print(A, B, A-B, A+B, 2*beta*(A-B)/(A+B))

0.0 0.0 0.0 0.0 nan


/Users/christopher/.anaconda/ipykernel/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  
